In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import mlflow
import numpy as np
import datetime
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lib.get_preprocess_fn import get_preprocess_fn
from lib import full_flow_dataloader
from lib.cross_validation import CustomKFoldCrossValidator, get_cross_validation_metrics, perform_cross_validation
from lib.metrics import rmse_metric, std_dev_metric
from lib.norms import Norm1Scaler, Norm3Scaler
from lib.reproduction import major_oxides

train_processed, test_processed = full_flow_dataloader.load_full_flow_data()

In [8]:
drop_cols = major_oxides + ["ID", "Sample Name"]
norm = 3

In [9]:
mlflow.set_experiment(f'XGBoost_Norm{norm}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')

models = []
rmse_scores = []

xgb_params = {
    "max_depth": 4,  # Slightly deeper trees since data is high-dimensional
    "min_child_weight": 5,  # Higher to control over-fitting
    "gamma": 0.1,  # Minimum loss reduction required to make further partition
    "subsample": 0.7,  # Subsample ratio of the training instances
    "colsample_bytree": 0.5,  # Subsample ratio of columns when constructing each tree
    "colsample_bylevel": 0.5,  # Subsample ratio of columns for each level
    "colsample_bynode": 0.5,  # Subsample ratio of columns for each split
    "lambda": 1,  # L2 regularization term on weights (lambda)
    "alpha": 0.5,  # L1 regularization term on weights (alpha)
    "learning_rate": 0.05,  # Step size shrinkage used in update to prevent overfitting
    "n_estimators": 100,  # Number of boosting rounds
    "objective": "reg:squarederror",  # Regression with squared loss
    "eval_metric": "rmse",  # Evaluation metric for validation data
}

for target in major_oxides:
    with mlflow.start_run(run_name=f"XGB_{target}"):
        # == Cross Validation ==
        kf = CustomKFoldCrossValidator(k=5, random_state=42, group_by="Sample Name")
        scaler = Norm1Scaler() if norm == 1 else Norm3Scaler()

        cv_metrics = perform_cross_validation(
            model=XGBRegressor(**xgb_params),
            preprocess_fn=get_preprocess_fn(target_col=target, drop_cols=drop_cols, preprocessor=scaler),
            kf=kf,
            data=train_processed,
            metric_fns=[rmse_metric, std_dev_metric],
        )

        mlflow.log_metrics(get_cross_validation_metrics(cv_metrics).as_dict())

        # == Training ==
        preprocess_fn = get_preprocess_fn(target_col=target, drop_cols=drop_cols, preprocessor=scaler)
        X_train, y_train, X_test, y_test = preprocess_fn(train_processed, test_processed)

        # Log hyperparameters
        mlflow.log_params(xgb_params)

        # Train the model
        model = XGBRegressor(**xgb_params)
        model.fit(X_train, y_train)
        models.append(model)

        pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, pred))

        mlflow.log_metric("rmse", float(rmse))


2024/05/03 10:03:39 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost_Norm3_20240503-100339' does not exist. Creating a new experiment.
